# Final Food Delivery Dataset

All transformations and analyses are executed using **PostgreSQL SQL**, while **Pandas** is used for result inspection and reporting.

---

## 1. Environment Setup

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text


## 2. PostgreSQL Connection

In [10]:
# Update credentials before running
username = 'postgres'
password = '12345'
host = 'localhost'
port = '5432'
database = 'food_delivery_db'

engine = create_engine(
    f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'
)

print('Connected to PostgreSQL')

Connected to PostgreSQL


## 3. Table Creation

In [11]:
create_tables_sql = '''
CREATE TABLE IF NOT EXISTS restaurants (
    restaurant_id INT,
    restaurant_name TEXT,
    cuisine TEXT,
    rating FLOAT
);

CREATE TABLE IF NOT EXISTS orders (
    order_id INT,
    user_id INT,
    restaurant_id INT,
    order_date TIMESTAMP,
    order_amount NUMERIC,
    restaurant_name VARCHAR(60)
);

CREATE TABLE IF NOT EXISTS users (
    user_id INT,
    name TEXT,
    city TEXT,
    membership TEXT
);
'''

with engine.begin() as conn:
    conn.execute(text(create_tables_sql))

print('Tables created successfully')

Tables created successfully


## 4. Load Data (CSV → PostgreSQL)

In [17]:
# Load CSV files
orders_df = pd.read_csv('orders.csv')
users_df = pd.read_csv('users.csv')

# Read restaurants table from PostgreSQL
restaurants_df = pd.read_sql(
    "SELECT * FROM restaurants",
    engine
)

# Load data into PostgreSQL
orders_df.to_sql('orders', engine, if_exists='replace', index=False)
users_df.to_sql('users', engine, if_exists='replace', index=False)
restaurants_df.to_sql('restaurants', engine, if_exists='replace', index=False)

print('Orders, Users and Restaurants data loaded successfully')


Orders, Users and Restaurants data loaded successfully


## 5. Verify Tables

In [13]:
pd.read_sql('SELECT * FROM restaurants LIMIT 5', engine)

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


## 6. Create Final Dataset (LEFT JOIN)

In [19]:
final_table_sql = '''
DROP TABLE IF EXISTS final_food_delivery_dataset;

CREATE TABLE final_food_delivery_dataset AS
SELECT 
    o.order_id,
    o.order_date,
    o.total_amount AS order_amount,
    u.user_id,
    u.name AS user_name,
    u.city,
    u.membership,
    r.restaurant_id,
    r.restaurant_name,
    r.cuisine,
    r.rating
FROM orders o
LEFT JOIN users u 
    ON o.user_id = u.user_id
LEFT JOIN restaurants r 
    ON o.restaurant_id = r.restaurant_id;

'''

with engine.begin() as conn:
    conn.execute(text(final_table_sql))

print('Final dataset created')

Final dataset created


## 7. Order Trend Analysis

In [22]:
query = '''
SELECT DATE_TRUNC('month', order_date::timestamp) AS month,
       COUNT(*) AS total_orders
FROM final_food_delivery_dataset
GROUP BY month
ORDER BY month;
'''

pd.read_sql(query, engine)

,month,total_orders
0,2023-01-01,804
1,2023-02-01,785
2,2023-03-01,903
3,2023-04-01,812
4,2023-05-01,844
5,2023-06-01,784
6,2023-07-01,859
7,2023-08-01,851
8,2023-09-01,812
9,2023-10-01,863


## 8. User Behavior Analysis

In [28]:
query = '''
SELECT user_id,
       COUNT(order_id) AS total_orders,
       SUM(order_amount::numeric) AS total_spend,
       ROUND(AVG(order_amount::numeric), 2) AS avg_order_value
FROM final_food_delivery_dataset
GROUP BY user_id
ORDER BY total_orders DESC;
'''

df = pd.read_sql(query, engine)
print(df)

      user_id  total_orders  total_spend  avg_order_value
0        2973            13      9063.07           697.16
1        1515            12     11556.49           963.04
2        1337            11      8907.50           809.77
3         496            11      9634.30           875.85
4         874            11      8916.55           810.60
...       ...           ...          ...              ...
2878     2964             1       459.32           459.32
2879      270             1       611.65           611.65
2880      525             1       765.49           765.49
2881     2018             1       128.37           128.37
2882      967             1       149.89           149.89

[2883 rows x 4 columns]


## 9. Membership Analysis

In [30]:
query = '''
SELECT membership,
       COUNT(order_id) AS total_orders,
       SUM(order_amount::numeric) AS revenue,
       ROUND(AVG(order_amount::numeric), 2) AS avg_order_value
FROM final_food_delivery_dataset
GROUP BY membership;
'''

df = pd.read_sql(query, engine)
print(df)

  membership  total_orders     revenue  avg_order_value
0    Regular          5013  4036259.23           805.16
1       Gold          4987  3975364.89           797.15


## 10. City & Cuisine Performance

In [31]:
query = '''
SELECT city, cuisine, SUM(order_amount) AS revenue
FROM final_food_delivery_dataset
GROUP BY city, cuisine
ORDER BY revenue DESC;
'''

pd.read_sql(query, engine)

,city,cuisine,revenue
0,Bangalore,Mexican,571004.61
1,Bangalore,Italian,567881.80
2,Bangalore,Indian,543014.04
3,Chennai,Mexican,534804.60
4,Bangalore,Chinese,525046.13
5,Chennai,Italian,522051.88
6,Pune,Indian,519241.56
7,Hyderabad,Mexican,500085.51
8,Chennai,Chinese,491710.42
9,Pune,Italian,482261.30


## 11. Revenue Distribution

In [33]:
query = '''
SELECT city,
ROUND(SUM(order_amount::numeric)*100 / (SELECT SUM(order_amount::numeric)
FROM final_food_delivery_dataset),2) AS revenue_percentage
FROM final_food_delivery_dataset
GROUP BY city
ORDER BY revenue_percentage DESC;
'''

pd.read_sql(query, engine)

,city,revenue_percentage
0,Bangalore,27.55
1,Chennai,24.85
2,Pune,24.03
3,Hyderabad,23.58


## 12. Seasonality Analysis

In [34]:
query = '''
SELECT DATE_TRUNC('month', order_date::timestamp) AS month,
       SUM(order_amount::numeric) AS monthly_revenue,
       COUNT(order_id) AS total_orders
FROM final_food_delivery_dataset
GROUP BY month
ORDER BY month;
'''

pd.read_sql(query, engine)

,month,monthly_revenue,total_orders
0,2023-01-01,645818.76,804
1,2023-02-01,630867.40,785
2,2023-03-01,716738.98,903
3,2023-04-01,666097.18,812
4,2023-05-01,668428.61,844
5,2023-06-01,610822.93,784
6,2023-07-01,688559.45,859
7,2023-08-01,694987.58,851
8,2023-09-01,653838.07,812
9,2023-10-01,699187.73,863
